In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import pickle as pk
import csv
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import re
import time
import warnings
warnings.filterwarnings('ignore')

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_X_op = pd.read_csv('D:/transaction_risk_competition/operation_train_new.csv')
train_X_op.columns

Index(['UID', 'day', 'mode', 'success', 'time', 'os', 'version', 'device1',
       'device2', 'device_code1', 'device_code2', 'device_code3', 'mac1',
       'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub'],
      dtype='object')

In [3]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
#pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [4]:
len_count=len(train_X_op['UID'])
for col in ['UID','day','time','mode','success','os','version','device_code1','device_code2','device_code3','device1','device2','mac1','mac2','ip1','ip1_sub','ip2','ip2_sub','wifi','geo_code']:
    print('{0}'.format(col),  '%.2f' %(train_X_op[col].isnull().sum()/len_count))

UID 0.00
day 0.00
time 0.00
mode 0.00
success 0.07
os 0.00
version 0.17
device_code1 0.28
device_code2 0.30
device_code3 0.89
device1 0.17
device2 0.29
mac1 0.90
mac2 0.39
ip1 0.18
ip1_sub 0.18
ip2 0.91
ip2_sub 0.91
wifi 0.69
geo_code 0.34


In [5]:
train_X_op=train_X_op[['UID','day','time','mode','success','os','version','device_code1','device_code2','device1','device2','mac2','ip1','ip1_sub','wifi','geo_code']]

In [6]:
train_X_op.shape

(1460843, 16)

In [7]:
train_X_op['mode'].value_counts()

c8741ce15ceac2a4    800059
d25caee90b27fa9b    184574
79f8b86938180c3c     82435
b501fa4fc58206b9     59132
acfaded7e04e7ba0     51041
072eee5c88d380df     27494
3b02bcd4b685d7a8     24008
00094ae2a1d62504     22750
963bd8a75ff9ab37     22169
20a91b45ef8f8221     20231
e1d2fad8ffdc66b3     18631
cd75f033b0488ace     17608
1c341176507fbd9b     11892
2f3e878175e34d9c     11824
cca82161b59127c1     11401
6b2aa6745680f08a     10648
bf79b3647c0878eb      9993
d21cae2b73884a8d      7735
479617cd802ed152      7346
08017d2cb28c2348      5100
e806d126013b42d1      4801
09080b31b40d57e8      4209
ac63e881c4e19402      3769
6440f87037199039      3272
37031ef728f5c886      3184
9c69742a831d6214      2799
dd06c9d2ea0a3750      2788
4c3d45ed38eefb7a      2782
745961ec187394cd      2758
8c7ce5ff939e66d1      2703
                     ...  
290ead084a839c02        47
ab37f730c615850f        38
a3f0d631c4fcaf38        37
12845a3fe90eb1de        27
8e463287d7146285        25
f4b0872cd482e33f        22
d

In [8]:
a=train_X_op['mode'].value_counts().index.tolist()
b=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]+([-1]*(len(a)-15))
dicts=dict(zip(a,b))
train_X_op['mode']=train_X_op['mode'].map(dicts)
del a,b,dicts

In [9]:
train_X_op['success'].value_counts()

1.0    1288920
0.0      70843
Name: success, dtype: int64

In [10]:
train_X_op['os'].value_counts()

102    1050731
200     240978
103     159508
101       8168
104        964
107        490
105          4
Name: os, dtype: int64

In [11]:
train_X_op['version'].value_counts()

7.0.9             488038
7.0.5             460673
7.0.2              97315
7.0.7              80362
6.6.2              32066
7.0.0              25322
6.6.3              10442
4.1.7               7573
7.0.1               1929
5.8.24              1500
7.1.0                761
6.5.0                729
5.8.21               529
1.2.0                373
1.0.0                286
6.0.4                167
6.1.0                127
5.8.15               109
7.0.6                105
5.8.23                60
5.8.20                59
5.8.11                46
6.6.0                 39
5.8.18                24
1.1.0                 20
7.0.8                 16
5.8.22                14
6.5.7                  9
6.5.5                  8
6.5.3                  7
0.0.2                  6
6.5.2                  5
6.5.4                  3
3.9.3                  2
5.1.1                  2
4.2.2                  1
4.3.0                  1
4.2.18.0522.r2         1
Name: version, dtype: int64

In [12]:
a=train_X_op['version'].value_counts().index.tolist()
b=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]+([-1]*(len(a)-15))
dicts=dict(zip(a,b))
train_X_op['version']=train_X_op['version'].map(dicts)
del a,b,dicts

In [13]:
train_X_op['wifi'].value_counts()

a3fd93fe06a59074    2221
6ec718bbccbaedd2    1611
c90ccafafe961fb1    1187
2b170a64f5473601    1123
253c450edf124226     965
0201faa26cdf62a2     947
d2eb6cf107ce2484     914
3d8a206e19c98381     900
d74b52f9f6a8b8f5     897
af1c336e05b3df4e     815
2109917457c421d3     767
4dbcdb498715e897     742
2bd6620b2ea0f049     736
89efed0cc9374acf     729
2c0f832d5d75448c     728
d6632db83058a448     690
a33bf58feacb6465     649
84a12b07964b9892     645
2eee99d8e6924c36     643
357326becd8cd77f     632
30d7785c331fd9ed     597
1fce8ba8cb6ee46a     580
10fec7cebb075b99     557
052b542d2cd5c5f7     532
5f438a6b5396b02b     522
f250971823029422     520
f4632e129659c3f1     513
df91b1a900a1a859     510
37ad8b859cb7e6b2     507
47c22d9996785c89     506
                    ... 
a2e1f4c2d340f5c0       1
c1a416f768e56526       1
4b5c3b05f80a7b36       1
727e87285eb1d8e5       1
da79c303c64665d8       1
f29caf4875871046       1
b1bca85e65fbe190       1
9a69b793473fb1b4       1
dc64de0b7d9323e0       1


In [14]:
### 可分组的字段uid ,mode,success,os,version

In [15]:
train_X_op.fillna(0,inplace = True)

,UID,mode_count,success_count,os_count,version_count,device_code1_count,device_code2_count,device1_count,device2_count,mac2_count,ip1_count,ip1_sub_count,geo_code_count
0,10000,9,9,9,9,9,9,9,9,9,9,9,9
1,10001,67,67,67,67,67,67,67,67,67,67,67,67
2,10002,11,11,11,11,11,11,11,11,11,11,11,11
3,10003,15,15,15,15,15,15,15,15,15,15,15,15
4,10004,34,34,34,34,34,34,34,34,34,34,34,34


In [57]:
train_X_op_uid_field1 = train_X_op[['UID', 'mode']]
train_X_op_uid_field1.drop_duplicates(['UID', 'mode'], 'first', inplace=True)
train_X_op_uid_field1_count = train_X_op_uid_field1[['mode']].groupby(train_X_op_uid_field1['UID']).count().add_suffix('_count_op').reset_index()
train_X_op_uid_field1_count.head(5)

,UID,mode_count
0,10000,3
1,10001,6
2,10002,4
3,10003,2
4,10004,7


In [58]:
for col in ['success','os','version','device_code1','device_code2','device1','device2','mac2','ip1','ip1_sub','geo_code','wifi',]:
    train_X_op_uid_field1_temp = train_X_op[['UID', '{0}'.format(col)]]
    train_X_op_uid_field1_temp.drop_duplicates(['UID', '{0}'.format(col)], 'first', inplace=True)
    train_X_op_uid_field1_temp_count = train_X_op_uid_field1_temp[['{0}'.format(col)]].groupby(train_X_op_uid_field1_temp['UID']).count().add_suffix('_count_op').reset_index()
    train_X_op_uid_field1_count = pd.merge(train_X_op_uid_field1_count, train_X_op_uid_field1_temp_count, how='left', on='UID')
train_X_op_uid_field1_count.head(5)

,UID,mode_count,success_count,os_count,version_count,device_code1_count,device_code2_count,device1_count,device2_count,mac2_count,ip1_count,ip1_sub_count,geo_code_count,wifi_count
0,10000,3,1,1,2,1,1,1,2,1,2,2,1,1
1,10001,6,2,3,2,2,3,3,2,2,10,10,4,2
2,10002,4,1,2,1,2,2,2,2,3,2,2,2,1
3,10003,2,1,1,1,1,1,1,2,1,1,1,1,2
4,10004,7,2,1,2,1,2,1,2,2,6,6,2,4


In [ ]:
#可分组的字段uid ,mode,success,os,version
##uid  mode

In [46]:
train_X_op_uid_field2_count=train_X_op.groupby(['UID','mode'])['mode'].count().unstack()
train_X_op_uid_field2_count.reset_index(inplace=True)
train_X_op_uid_field2_count.columns=["UID"]+[ "mode"+"_"+str(i) for i in train_X_op_uid_field2_count.columns[1:]]
train_X_op_uid_field2_count.head(5)
train_X_op_uid_field2_count.fillna(0.0,inplace=True)
train_X_op_uid_field2_count.shape

for col in ["success","os","version"]:
    train_X_op_uid_filed2_count_temp=train_X_op.groupby(['UID','{0}'.format(col)])['{0}'.format(col)].count().unstack()
    train_X_op_uid_filed2_count_temp.reset_index(inplace=True)
    train_X_op_uid_filed2_count_temp.columns=["UID"]+[ '{0}'.format(col)+"_"+str(i) for i in train_X_op_uid_filed2_count_temp.columns[1:]]
    train_X_op_uid_filed2_count_temp.fillna(0.0,inplace=True)
    train_X_op_uid_field2_count = pd.merge(train_X_op_uid_field2_count, train_X_op_uid_filed2_count_temp, how='left', on='UID')
train_X_op_uid_field2_count.head(5)
    
    


,UID,mode_-1,mode_0,mode_1,mode_2,mode_3,mode_4,mode_5,mode_6,mode_7,mode_8,mode_9,mode_10,mode_11,mode_12,mode_13,mode_14,success_0.0,success_1.0,os_101,os_102,os_103,os_104,os_105,os_107,os_200,version_-1.0,version_0.0,version_1.0,version_2.0,version_3.0,version_4.0,version_5.0,version_6.0,version_7.0,version_8.0,version_9.0,version_10.0,version_11.0,version_12.0,version_13.0,version_14.0
0,10000,1.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,34.0,9.0,17.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,18.0,49.0,0.0,49.0,0.0,1.0,0.0,0.0,17.0,0.0,19.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,4.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,4.0,21.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,32.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
train_X_op_uid_day = train_X_op[['UID', 'day']]
train_X_op_uid_day.drop_duplicates(['UID', 'day'], 'first', inplace=True)
train_X_op_uid_day_count = train_X_op_uid_day[['day']].groupby(train_X_op_uid_day['UID']).count().add_suffix('_count').reset_index()
train_X_op_uid_field2_count = pd.merge(train_X_op_uid_field2_count, train_X_op_uid_day_count, how='left', on='UID')
train_X_op_uid_field2_count["success_1.0_ration"]=train_X_op_uid_field2_count["success_1.0_ration"]/(train_X_op_uid_field2_count["success_0.0"]+train_X_op_uid_field2_count["success_1.0"])
train_X_op_uid_field2_count.head(5)

,UID,mode_-1,mode_0,mode_1,mode_2,mode_3,mode_4,mode_5,mode_6,mode_7,mode_8,mode_9,mode_10,mode_11,mode_12,mode_13,mode_14,success_0.0,success_1.0,os_101,os_102,os_103,os_104,os_105,os_107,os_200,version_-1.0,version_0.0,version_1.0,version_2.0,version_3.0,version_4.0,version_5.0,version_6.0,version_7.0,version_8.0,version_9.0,version_10.0,version_11.0,version_12.0,version_13.0,version_14.0,day_count
0,10000,1.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,10001,0.0,34.0,9.0,17.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,18.0,49.0,0.0,49.0,0.0,1.0,0.0,0.0,17.0,0.0,19.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
2,10002,4.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,10003,0.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,10004,4.0,21.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,32.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6


Index(['UID', 'mode_-1', 'mode_0', 'mode_1', 'mode_2', 'mode_3', 'mode_4',
       'mode_5', 'mode_6', 'mode_7', 'mode_8', 'mode_9', 'mode_10', 'mode_11',
       'mode_12', 'mode_13', 'mode_14', 'success_0.0', 'success_1.0', 'os_101',
       'os_102', 'os_103', 'os_104', 'os_105', 'os_107', 'os_200',
       'version_-1.0', 'version_0.0', 'version_1.0', 'version_2.0',
       'version_3.0', 'version_4.0', 'version_5.0', 'version_6.0',
       'version_7.0', 'version_8.0', 'version_9.0', 'version_10.0',
       'version_11.0', 'version_12.0', 'version_13.0', 'version_14.0',
       'day_count'],
      dtype='object')

In [49]:
for col in train_X_op_uid_field2_count.columns.tolist()[1:-1]:
    train_X_op_uid_field2_count["{0}_day_per".format(col)]=train_X_op_uid_field2_count["{0}".format(col)]/train_X_op_uid_field2_count["day_count"]
    
train_X_op_uid_field2_count.head(5)

,UID,mode_-1,mode_0,mode_1,mode_2,mode_3,mode_4,mode_5,mode_6,mode_7,mode_8,mode_9,mode_10,mode_11,mode_12,mode_13,mode_14,success_0.0,success_1.0,os_101,os_102,os_103,os_104,os_105,os_107,os_200,version_-1.0,version_0.0,version_1.0,version_2.0,version_3.0,version_4.0,version_5.0,version_6.0,version_7.0,version_8.0,version_9.0,version_10.0,version_11.0,version_12.0,version_13.0,version_14.0,day_count,mode_-1_day_per,mode_0_day_per,mode_1_day_per,mode_2_day_per,mode_3_day_per,mode_4_day_per,mode_5_day_per,mode_6_day_per,mode_7_day_per,mode_8_day_per,mode_9_day_per,mode_10_day_per,mode_11_day_per,mode_12_day_per,mode_13_day_per,mode_14_day_per,success_0.0_day_per,success_1.0_day_per,os_101_day_per,os_102_day_per,os_103_day_per,os_104_day_per,os_105_day_per,os_107_day_per,os_200_day_per,version_-1.0_day_per,version_0.0_day_per,version_1.0_day_per,version_2.0_day_per,version_3.0_day_per,version_4.0_day_per,version_5.0_day_per,version_6.0_day_per,version_7.0_day_per,version_8.0_day_per,version_9.0_day_per,version_10.0_day_per,version_11.0_day_per,version_12.0_day_per,version_13.0_day_per,version_14.0_day_per
0,10000,1.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.500000,3.00,1.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,4.500000,0.0,0.000000,4.5,0.000,0.0,0.0,0.000,0.0,0.000000,2.000000,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,34.0,9.0,17.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,18.0,49.0,0.0,49.0,0.0,1.0,0.0,0.0,17.0,0.0,19.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0.000000,4.25,1.125000,2.125,0.5,0.0,0.000000,0.0,0.0,0.125000,0.0,0.0,0.0,0.25,0.0,0.0,2.250000,6.125000,0.0,6.125000,0.0,0.125,0.0,0.0,2.125,0.0,2.375000,0.000000,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,4.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4.000000,3.00,3.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.00,0.0,0.0,0.000000,11.000000,1.0,10.000000,0.0,0.000,0.0,0.0,0.000,0.0,11.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000,14.00,1.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,15.000000,0.0,15.000000,0.0,0.000,0.0,0.0,0.000,0.0,15.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,4.0,21.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,32.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,0.666667,3.50,0.166667,0.000,0.5,0.5,0.166667,0.0,0.0,0.166667,0.0,0.0,0.0,0.00,0.0,0.0,0.333333,5.333333,0.0,5.666667,0.0,0.000,0.0,0.0,0.000,0.0,2.333333,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
train_X_op_uid_field1_count = pd.merge(train_X_op_uid_field1_count, train_X_op_uid_day_count, how='left', on='UID')
train_X_op_uid_field1_count.head(5)

,UID,mode_count,success_count,os_count,version_count,device_code1_count,device_code2_count,device1_count,device2_count,mac2_count,ip1_count,ip1_sub_count,geo_code_count,wifi_count,day_count
0,10000,3,1,1,2,1,1,1,2,1,2,2,1,1,2
1,10001,6,2,3,2,2,3,3,2,2,10,10,4,2,8
2,10002,4,1,2,1,2,2,2,2,3,2,2,2,1,1
3,10003,2,1,1,1,1,1,1,2,1,1,1,1,2,1
4,10004,7,2,1,2,1,2,1,2,2,6,6,2,4,6


In [60]:
for col in train_X_op_uid_field1_count.columns.tolist()[1:-1]:
    train_X_op_uid_field1_count["{0}_day_per".format(col)]=train_X_op_uid_field1_count["{0}".format(col)]/train_X_op_uid_field2_count["day_count"]
    
train_X_op_uid_field1_count.head(5)

,UID,mode_count,success_count,os_count,version_count,device_code1_count,device_code2_count,device1_count,device2_count,mac2_count,ip1_count,ip1_sub_count,geo_code_count,wifi_count,day_count,mode_count_day_per,success_count_day_per,os_count_day_per,version_count_day_per,device_code1_count_day_per,device_code2_count_day_per,device1_count_day_per,device2_count_day_per,mac2_count_day_per,ip1_count_day_per,ip1_sub_count_day_per,geo_code_count_day_per,wifi_count_day_per
0,10000,3,1,1,2,1,1,1,2,1,2,2,1,1,2,1.500000,0.500000,0.500000,1.000000,0.500000,0.500000,0.500000,1.000000,0.500000,1.00,1.00,0.500000,0.500000
1,10001,6,2,3,2,2,3,3,2,2,10,10,4,2,8,0.750000,0.250000,0.375000,0.250000,0.250000,0.375000,0.375000,0.250000,0.250000,1.25,1.25,0.500000,0.250000
2,10002,4,1,2,1,2,2,2,2,3,2,2,2,1,1,4.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.00,2.00,2.000000,1.000000
3,10003,2,1,1,1,1,1,1,2,1,1,1,1,2,1,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.00,1.00,1.000000,2.000000
4,10004,7,2,1,2,1,2,1,2,2,6,6,2,4,6,1.166667,0.333333,0.166667,0.333333,0.166667,0.333333,0.166667,0.333333,0.333333,1.00,1.00,0.333333,0.666667


In [64]:
feature_op_train = pd.merge(train_X_op_uid_field1_count, train_X_op_uid_field2_count, how='left', on='UID')

In [65]:
feature_op_train.shape

(29728, 112)